In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install --upgrade tensorflow-hub

Requirement already up-to-date: tensorflow-hub in /usr/local/lib/python3.6/dist-packages (0.10.0)


In [3]:
!pip install spektral

     |████████████████████████████████| 112kB 9.1MB/s 


In [1]:
import numpy as np
import tensorflow as tf
import argparse
import scipy.sparse as sp
import pandas as pd 
import random
import json
import os

# from feature_matrix import FeatureMatrix

import pandas as pd 
import tensorflow_hub as hub
from bert import run_classifier
from bert import tokenization


In [2]:
##Code referenced from: 
##Website Title: BERT in Keras with Tensorflow hub
##URL : https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b

def create_tokenizer_from_hub_module():
  with tf.Graph().as_default():
    bert_module = hub.Module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.compat.v1.Session() as sess:
        vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                              tokenization_info["do_lower_case"]])

  return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [3]:
    def getFeatures( dataset="BuzzFeed"):
        feature_df =  FM.get_feature_matrix(dataset)
        label = feature_df['label'].tolist()
        label_comp = [0 if each else 1 for each in label]
        global label_zip
        label_zip = list(zip(label_comp, label))
        feature_df.drop(['label'], axis=1)
        feature_np = feature_df.values

        return sp.csr_matrix(feature_np, dtype=float).tolil()

In [4]:
def getYs():

  random.seed(1)
  yTrain =  label_zip[:]
  yVal =  label_zip[:]
  yTest =  label_zip[:]
  train_mask = [False] * len(yTrain)
  val_mask = [False] * len(yTrain)
  test_mask = [False] * len(yTrain)
  n = len(yTrain)

  set_of_records_range = set(range(n))

  train_range = set(random.sample(set_of_records_range, k=int(n * 0.6)))
  set_of_records_range = set_of_records_range - train_range

  val_range = set(random.sample(set_of_records_range, k=int(n * 0.2)))
  set_of_records_range = set_of_records_range - train_range

  test_range = set(random.sample(set_of_records_range, k=int(n * 0.2)))

  for i in train_range:
      yVal[i] = (0,0)
      yTest[i] = (0,0)
      train_mask[i] = True
  for i in val_range:
      yTrain[i] = (0,0)
      yTest[i] = (0,0)
      val_mask[i] = True
  for i in test_range:
      yVal[i] = (0,0)
      yTrain[i] = (0,0)
      test_mask[i] = True

  return yTrain, yVal, yTest, train_mask, val_mask, test_mask


In [5]:
def sparse_to_tuple(sparse_mx):
  """Convert sparse matrix to tuple representation."""
  def to_tuple(mx):
    if not sp.isspmatrix_coo(mx):
        mx = mx.tocoo()
    coords = np.vstack((mx.row, mx.col)).transpose()
    values = mx.data
    shape = mx.shape
    return coords, values, shape

  if isinstance(sparse_mx, list):
    for i in range(len(sparse_mx)):
        sparse_mx[i] = to_tuple(sparse_mx[i])
  else:
    sparse_mx = to_tuple(sparse_mx)

  return sparse_mx

In [6]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1, dtype=float).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features.todense(), sparse_to_tuple(features)

In [7]:
base_path = "/content/drive/MyDrive/Colab_Notebooks/DL_final/"
folder = "BuzzFeed"
data_list = []
for subfolder in ["FakeNewsContent", "RealNewsContent"]:
  print("Getting data from subfolder: ", subfolder)
  file_array = [f for f in os.listdir(base_path + folder + "/" + subfolder) if f.endswith('.json')]
  file_array.sort() # file is sorted list
  file_array = [os.path.join(base_path + folder + "/" + subfolder, name) for name in file_array]

  for file in file_array:
    # print("file: ", file)
    # print("path: ",base_path + folder + "/" + subfolder)
    with open(file, 'r') as json_file:
      data = json.load(json_file)
      if file.split("/")[-2] == "FakeNewsContent":
          data_list.append([data['text'], 1])
      else:
          data_list.append([data['text'], 0])

print("Creating data frame")
data_frame = pd.DataFrame(data_list, columns=["text", "label"])
data_frame = data_frame.sample(frac=1)

input = data_frame.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                       text_a=x['text'], text_b=None, label=x['label']), axis=1)
print(data_frame)
print("extracting features")
tokenizer = create_tokenizer_from_hub_module()
features = run_classifier.convert_examples_to_features(input, [0, 1], 128, tokenizer)

train_features_list = []
for item in features:
  temp = item.input_ids
  temp.append(item.label_id)
  train_features_list.append(temp)
column_names = ["feature" + str(i) for i in range(128)]
column_names.append("label")
features_frame = pd.DataFrame(train_features_list, columns=column_names)  ##extracted features data frame
# /content/drive/MyDrive/Colab_Notebooks/DL_final/BuzzFeed/FakeNewsContent/BuzzFeed_Fake_3-Webpage.json

label = features_frame['label'].tolist()
label_comp = [0 if each else 1 for each in label]
global label_zip
label_zip = list(zip(label_comp, label))
features_frame.drop(['label'], axis=1)
feature_np = features_frame.values

Getting data from subfolder:  FakeNewsContent
Getting data from subfolder:  RealNewsContent
Creating data frame
                                                  text  label
98   Peaceful protesters crowded Charlotte's first ...      0
141  Kellyanne Conway praised Donald Trump for show...      0
156  Under the Radar Blog Archives Select Date… Jul...      0
92   Less than a day after protests over the police...      0
20   A Daisy of a Rerun\n\nI am of an age which inc...      1
..                                                 ...    ...
114  Obama wears hat, breaking ‘Politics 101’ rule ...      0
128  7.5k SHARES Facebook Twitter\n\nA public high ...      0
56   71k SHARES SHARE THIS STORY\n\nNew Jersey’s de...      1
17   What would possess anyone to take control of t...      1
19   In every state there is a law governing the me...      1

[182 rows x 2 columns]
extracting features
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 182


INFO:tensorflow:Writing example 0 of 182


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


In [8]:
features =  sp.csr_matrix(feature_np, dtype=float).tolil()
adj =  pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/DL_final/news_news_bf_adjacency_matrix.csv", header=None).values
y_train, y_val, y_test, train_mask, val_mask, test_mask =  getYs()

features=sp.csr_matrix(features)

print(features)
features, sparse_matrix = preprocess_features(features)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label = label_encoder.fit_transform(label)
label = tf.keras.utils.to_categorical(label)

  (0, 0)	101.0
  (0, 1)	9379.0
  (0, 2)	13337.0
  (0, 3)	10789.0
  (0, 4)	5904.0
  (0, 5)	1005.0
  (0, 6)	1055.0
  (0, 7)	2034.0
  (0, 8)	2103.0
  (0, 9)	2473.0
  (0, 10)	3116.0
  (0, 11)	2144.0
  (0, 12)	1996.0
  (0, 13)	8872.0
  (0, 14)	5008.0
  (0, 15)	1997.0
  (0, 16)	6766.0
  (0, 17)	16983.0
  (0, 18)	12162.0
  (0, 19)	3660.0
  (0, 20)	1010.0
  (0, 21)	2007.0
  (0, 22)	2058.0
  (0, 23)	2753.0
  (0, 24)	2111.0
  :	:
  (181, 104)	1999.0
  (181, 105)	1996.0
  (181, 106)	2142.0
  (181, 107)	2163.0
  (181, 108)	1010.0
  (181, 109)	1998.0
  (181, 110)	2097.0
  (181, 111)	2644.0
  (181, 112)	2012.0
  (181, 113)	2498.0
  (181, 114)	1010.0
  (181, 115)	2164.0
  (181, 116)	14303.0
  (181, 117)	9861.0
  (181, 118)	1010.0
  (181, 119)	10428.0
  (181, 120)	28652.0
  (181, 121)	1010.0
  (181, 122)	1998.0
  (181, 123)	2151.0
  (181, 124)	2060.0
  (181, 125)	6206.0
  (181, 126)	2965.0
  (181, 127)	102.0
  (181, 128)	1.0


In [38]:
print(label)

[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 

In [9]:
y_train  = np.asarray(y_train)
y_val = np.asarray(y_val)
y_test = np.asarray(y_test)
train_mask  = np.asarray(train_mask)
val_mask  = np.asarray(val_mask)
test_mask  = np.asarray(test_mask)
label = np.asarray(label)

In [40]:
# np.isnan(y_train.data).any()
# np.isnan(y_val.data).any()
np.isnan(y_test.data).any()

False

In [41]:
train_mask.shape

(182,)

In [71]:
from __future__ import division
from __future__ import print_function
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l2
import time
import tensorflow as tf 
from spektral.layers import GCNConv
# from  utils import *
# from models import GCN, MLP

# Set random seed
seed = 123
np.random.seed(seed)
tf.random.set_seed(seed)

epochs = 200
dropout = 0.5
weight_decay = 2e-5
early_stopping = 10
learning_rate = 2e-5
num_nodes = features.shape[0]
# num_nodes = adj.shape[0]
l2_reg = 5e-4 / 2
num_classes = 2
# 'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),


In [63]:
features.shape[1]

129

##GCN1

In [63]:
adj_matrix = GCNConv.preprocess(adj).astype('f4')
adj_matrix[np.isinf(adj_matrix)] = 1

In [79]:

x_inputs = Input(shape = features.shape[1])
a_inputs = Input((num_nodes,), sparse=True, dtype=tf.float32)
do_1 = Dropout(dropout)(x_inputs)
gc_1 = GCNConv(64,
               activation='relu',
               kernel_regularizer=l2(l2_reg),
               use_bias=False)([do_1, a_inputs])
do_2 = Dropout(dropout)(gc_1)
gc_2 = GCNConv(64,
               activation='relu',
               kernel_regularizer=l2(l2_reg),
               use_bias=False)([do_2, a_inputs])

output =  Dense(units=num_classes, activation='softmax')(gc_2)
model = Model(inputs=[x_inputs, a_inputs], outputs=output)
optimizer = SGD(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()



Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 129)]        0                                            
__________________________________________________________________________________________________
dropout_12 (Dropout)            (None, 129)          0           input_15[0][0]                   
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 182)]        0                                            
__________________________________________________________________________________________________
gcn_conv_12 (GCNConv)           (None, 64)           8256        dropout_12[0][0]                 
                                                                 input_16[0][0]       

In [80]:
tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./Tensorboard_GCN_cora',
)
callback_GCN = [tbCallBack_GCN]

In [81]:


print(adj_matrix)
# x[numpy.isneginf(x)] = 0
adj_matrix[np.isinf(adj_matrix)] = 1
print(adj_matrix)
# print(max(adj_matrix[0]))

[[0.04651163 0.         0.         ... 0.01495371 0.         0.        ]
 [0.         0.03333334 0.016265   ... 0.         0.         0.        ]
 [0.         0.016265   0.03174603 ... 0.01235416 0.         0.        ]
 ...
 [0.01495371 0.         0.01235416 ... 0.01923077 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]]
[[0.04651163 0.         0.         ... 0.01495371 0.         0.        ]
 [0.         0.03333334 0.016265   ... 0.         0.         0.        ]
 [0.         0.016265   0.03174603 ... 0.01235416 0.         0.        ]
 ...
 [0.01495371 0.         0.01235416 ... 0.01923077 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]]


In [82]:
print('adj shape', type(adj))

print('features shape', type(features))
print('y_train shape', type(y_train))

print('y_val shape', type(y_val))
print('y_test shape', type(y_test))
print('train_mask shape', type(train_mask))
print('val_mask shape', type(val_mask))
print('test_mask shape', type(test_mask)) 
print('label', type(label))


adj shape <class 'numpy.ndarray'>
features shape <class 'numpy.matrix'>
y_train shape <class 'numpy.ndarray'>
y_val shape <class 'numpy.ndarray'>
y_test shape <class 'numpy.ndarray'>
train_mask shape <class 'numpy.ndarray'>
val_mask shape <class 'numpy.ndarray'>
test_mask shape <class 'numpy.ndarray'>
label <class 'numpy.ndarray'>


In [83]:
print(np.isnan(y_train).any())
print(np.isnan(y_val).any())
print(np.isnan(y_test).any())
print(np.isnan(train_mask).any())
print(np.isnan(val_mask).any())
print(np.isnan(test_mask).any())
print(np.isnan(label).any())
print(np.isnan(features).any())
print(np.isnan(adj_matrix).any())

False
False
False
False
False
False
False
False
False


In [84]:
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)


print(label.shape)

(182, 2)
(182, 2)
(182, 2)
(182, 2)


In [85]:
# Train model
validation_data = ([features, adj_matrix], label, val_mask)
model.fit([features, adj_matrix],
          label,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=num_nodes,
          validation_data=validation_data,
          shuffle=False,
          )


# callbacks=[
#               EarlyStopping(patience=early_stopping,  restore_best_weights=True),
#               tbCallBack_GCN
#           ]
# history = model.fit(
#     features, 
#     y_train, 
#     sample_weight=tf.cast(train_mask, tf.float32), # This will be used in loss calculations
#     validation_data=(features, y_val, val_mask),
#     epochs=epochs, 
#     batch_size=num_nodes, # This is unusual in ML - since our adjacency matrix is the whole graph, we want to feed in the whole node_state array in each training step
#     verbose=1,
#     shuffle=False # Do not shuffle the order of our input data, since its order matches up to the adjacency matrix
# )


Epoch 1/200
1/1 [==============================] - 0s 189ms/step - loss: 0.4527 - acc: 0.5138 - val_loss: 169490738778663408543475957760.0000 - val_acc: 0.4722
Epoch 2/200
1/1 [==============================] - 0s 21ms/step - loss: 0.4526 - acc: 0.5138 - val_loss: 169490757668129340022056812544.0000 - val_acc: 0.4722
Epoch 3/200
1/1 [==============================] - 0s 20ms/step - loss: 0.4526 - acc: 0.5138 - val_loss: 169490757668129340022056812544.0000 - val_acc: 0.4722
Epoch 4/200
1/1 [==============================] - 0s 19ms/step - loss: 0.4528 - acc: 0.5138 - val_loss: 169490757668129340022056812544.0000 - val_acc: 0.4722
Epoch 5/200
1/1 [==============================] - 0s 20ms/step - loss: 0.4527 - acc: 0.5138 - val_loss: 169490757668129340022056812544.0000 - val_acc: 0.4722
Epoch 6/200
1/1 [==============================] - 0s 21ms/step - loss: 0.4526 - acc: 0.5138 - val_loss: 169490757668129340022056812544.0000 - val_acc: 0.4722
Epoch 7/200
1/1 [============================

##GCN2

In [ ]:
from  utils import *
from models import GCN, MLP

flags = tf.compat.v1.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')
# flags.DEFINE_string('dataset', 'cora', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
# flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
# flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')



In [17]:
print(features[2])

[[0.00017975 0.00374449 0.00355228 0.01284408 0.01766706 0.01470743
  0.01294908 0.00544232 0.00272116 0.0040061  0.0017975  0.00430331
  0.00398296 0.0017975  0.00430331 0.00371779 0.0017975  0.00430331
  0.00401856 0.0017975  0.00430331 0.00397407 0.0017975  0.00430331
  0.00415915 0.0017975  0.00430331 0.00401144 0.0017975  0.00430331
  0.00406661 0.0017975  0.00419119 0.00405949 0.0017975  0.00419119
  0.00401322 0.0017975  0.00419119 0.00399364 0.0017975  0.00419119
  0.00401678 0.0017975  0.00419119 0.01918693 0.00883976 0.01950193
  0.00624852 0.00368753 0.00356474 0.00458984 0.00364126 0.01774003
  0.0036377  0.00184555 0.03256314 0.00357185 0.00464501 0.01232975
  0.00355762 0.03927082 0.0017975  0.00186868 0.00180105 0.00186156
  0.00180105 0.0018936  0.03094539 0.00404169 0.00184911 0.01611873
  0.03252932 0.00368753 0.00180283 0.00664005 0.0169712  0.00548325
  0.01727375 0.00188114 0.00180105 0.00187758 0.00180105 0.00872942
  0.01054827 0.00365372 0.00975453 0.0035594  0.

In [16]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
model_func = GCN

support = [preprocess_adj(adj)]
num_supports = 1
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0.2, shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}
# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()

# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)







IndexError: ignored

In [ ]:
# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    print('before feed')
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

In [ ]:
# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))